In [1]:
import pandas as pd
from IPython.display import display

# Load Excel file
# Define file path
file_path = "TestDataAnalyst.xlsx"

# Read the 'user' and 'transaction' tabs
user_df = pd.read_excel(file_path, sheet_name="User")
df = pd.read_excel(file_path, sheet_name="Transactions")


# Define the structure of the DataFrame for transactions
transaction_columns = ["Account ID", " Date", "Amount In", "Amount Out", "Balance", "Type", "Desc1|Desc2", "ExIf"]
transactions_df = pd.DataFrame(columns=transaction_columns)

# Function to parse transaction rows
# Assumes transaction rows follow the account row and have consistent columns
def parse_transaction_row(account_id, row):
    transaction = {"Account ID": account_id}  # Add the current account ID
    for col, value in zip(transaction_columns[1:], row):
        transaction[col] = value
    return transaction

# Function to populate the transaction DataFrame
def build_transactions_dataframe(transactions_raw):
    transactions = []  # List to store transaction dictionaries
    current_account_id = None

    for _, row in transactions_raw.iterrows():
        if isinstance(row[0], str) and row[0].startswith("Account ID"):
            # Extract the Account ID from the row
            current_account_id = row[0].split(", ")[0].split(": ", 1)[1].strip()    
        elif current_account_id:
            # Treat as a transaction row
            transaction = parse_transaction_row(current_account_id, row)
            transactions.append(transaction)

    return pd.DataFrame(transactions, columns=transaction_columns)

# Function to build the account details DataFrame
def build_accounts_dataframe(accounts_raw):
    accounts_df = pd.DataFrame(accounts_raw)
    
    accounts_df.columns = [col.strip() for col in accounts_df.iloc[14]]  # Use 15th row as column headers
    print(accounts_df.columns)
    accounts_df = accounts_df[15:]  # Drop the 15th row used as headers
    accounts_df = accounts_df.reset_index(drop=True)
    # display(accounts_df)
    return accounts_df

def join_dataframes(transactions_df, accounts_df):
    
    # Strip leading/trailing spaces and convert to consistent case
    # transactions_df['Account ID'] = transactions_df['Account ID'].str.strip()
    # accounts_df['Account ID'] = accounts_df['Account ID'].str.strip()

    # Ensure consistent data type
    transactions_df['Account ID'] = transactions_df['Account ID'].astype(str)
    accounts_df['Account ID'] = accounts_df['Account ID'].astype(str)
    
    display(accounts_df)
    display(transactions_df)
    merged_df = pd.merge(transactions_df, accounts_df, on="Account ID", how="inner")
    display(merged_df)
    return merged_df

# Save the transactions DataFrame to an Excel file
def save_to_excel(df, output_file):
    df.to_excel(output_file, index=False, engine="openpyxl")
    
# Example usage
# Assuming transactions_raw is a DataFrame containing rows of account and transaction data
transactions_raw = pd.read_excel(file_path, sheet_name="Transactions",engine="openpyxl", header=None)
transactions_df = build_transactions_dataframe(transactions_raw)

accounts_raw = pd.read_excel(file_path ,sheet_name="User",engine="openpyxl", header=None)
accounts_df = build_accounts_dataframe(accounts_raw)

final_df = join_dataframes(transactions_df, accounts_df)
save_to_excel(final_df, "Joined_database.xlsx")

# Display the merged DataFrame (Uncomment below for actual use)
# print(final_df)



Index(['Account ID', 'FI ID', 'Name', 'Number', 'Credit Limit',
       'Interest Rate', 'Joint', 'Status', 'Open', 'Closed', 'Known since',
       'Total Trxs', '2024-12', '2024-11', '2024-10', '2024-09', '2024-08',
       '2024-07'],
      dtype='object')


,Account ID,FI ID,Name,Number,Credit Limit,Interest Rate,Joint,Status,Open,Closed,Known since,Total Trxs,2024-12,2024-11,2024-10,2024-09,2024-08,2024-07
0,523319000,HSUFXNKONE6HHUIFNTLO-0001-Savings,Savings,1,$0.00,0.00%,False,O,2024-10-29 00:00:00,None,2024-10-29 00:00:00,53,$43.39,"$1,020.17",$26.72,$0.00,$0.00,$0.00
1,523319001,HSUFXNKONE6HHUIFNTLO-0002-Checking,MyDesign Checking,2,$0.00,0.00%,False,O,2024-10-29 00:00:00,None,2024-10-29 00:00:00,254,$-753.60,"$1,464.81","$4,931.18",$0.00,$0.00,$0.00
2,523319002,HSUFXNKONE6HHUIFNTLO-0003-Money Markets,Money Market Savings,3,$0.00,0.00%,False,O,2024-10-29 00:00:00,None,2024-10-29 00:00:00,1,$0.00,$0.00,$1.49,$0.00,$0.00,$0.00
3,523319003,HSUFXNKONE6HHUIFNTLO-0001-Mortgage Loans,Home Equity Line of Credit,1,$0.00,0.00%,False,O,2006-10-12 00:00:00,None,2006-10-12 00:00:00,11,$206.42,"$-1,898.32",$0.00,$0.00,$0.00,$0.00
4,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,No salaries during period: 2024-07 to 2025-01 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Account ID,Date,Amount In,Amount Out,Balance,Type,Desc1|Desc2,ExIf
0,523319000,Date,Amount In,Amount Out,Balance,Type,Desc1|Desc2 ...,ExIf ...
1,523319000,Start Bal,,,$0.00,,...,...
2,523319000,2024-10-29 00:00:00,$26.72,,$26.72,TransferOtherUser,|FROM ANONYMOUS 0000267285 SHARE 0001 ...,Received ...
3,523319000,2024-10-29 00:00:00,$1.49,,$28.21,TransferOtherUser,|FROM ANONYMOUS 0000267285 SHARE 0066 ...,Received ...
4,523319000,2024-10-29 00:00:00,,$-1.49,$26.72,,| ...,...
...,...,...,...,...,...,...,...,...
339,523319003,2024-12-31 00:00:00,,$-5.91,"$1,691.90",InterestsOnLoan,|,...
340,523319003,Total,$400.00,"$-2,091.90","$-1,691.90",,,...
341,523319003,NaN,NaN,NaN,NaN,NaN,NaN,NaN
342,523319003,Account Mapped Trxs 10 (90.91%),NaN,NaN,NaN,NaN,NaN,NaN


,Account ID,Date,Amount In,Amount Out,Balance,Type,Desc1|Desc2,ExIf,FI ID,Name,...,Open,Closed,Known since,Total Trxs,2024-12,2024-11,2024-10,2024-09,2024-08,2024-07
0,523319000,Date,Amount In,Amount Out,Balance,Type,Desc1|Desc2 ...,ExIf ...,HSUFXNKONE6HHUIFNTLO-0001-Savings,Savings,...,2024-10-29 00:00:00,None,2024-10-29 00:00:00,53,$43.39,"$1,020.17",$26.72,$0.00,$0.00,$0.00
1,523319000,Start Bal,,,$0.00,,...,...,HSUFXNKONE6HHUIFNTLO-0001-Savings,Savings,...,2024-10-29 00:00:00,None,2024-10-29 00:00:00,53,$43.39,"$1,020.17",$26.72,$0.00,$0.00,$0.00
2,523319000,2024-10-29 00:00:00,$26.72,,$26.72,TransferOtherUser,|FROM ANONYMOUS 0000267285 SHARE 0001 ...,Received ...,HSUFXNKONE6HHUIFNTLO-0001-Savings,Savings,...,2024-10-29 00:00:00,None,2024-10-29 00:00:00,53,$43.39,"$1,020.17",$26.72,$0.00,$0.00,$0.00
3,523319000,2024-10-29 00:00:00,$1.49,,$28.21,TransferOtherUser,|FROM ANONYMOUS 0000267285 SHARE 0066 ...,Received ...,HSUFXNKONE6HHUIFNTLO-0001-Savings,Savings,...,2024-10-29 00:00:00,None,2024-10-29 00:00:00,53,$43.39,"$1,020.17",$26.72,$0.00,$0.00,$0.00
4,523319000,2024-10-29 00:00:00,,$-1.49,$26.72,,| ...,...,HSUFXNKONE6HHUIFNTLO-0001-Savings,Savings,...,2024-10-29 00:00:00,None,2024-10-29 00:00:00,53,$43.39,"$1,020.17",$26.72,$0.00,$0.00,$0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,523319003,2024-12-31 00:00:00,,$-5.91,"$1,691.90",InterestsOnLoan,|,...,HSUFXNKONE6HHUIFNTLO-0001-Mortgage Loans,Home Equity Line of Credit,...,2006-10-12 00:00:00,None,2006-10-12 00:00:00,11,$206.42,"$-1,898.32",$0.00,$0.00,$0.00,$0.00
340,523319003,Total,$400.00,"$-2,091.90","$-1,691.90",,,...,HSUFXNKONE6HHUIFNTLO-0001-Mortgage Loans,Home Equity Line of Credit,...,2006-10-12 00:00:00,None,2006-10-12 00:00:00,11,$206.42,"$-1,898.32",$0.00,$0.00,$0.00,$0.00
341,523319003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HSUFXNKONE6HHUIFNTLO-0001-Mortgage Loans,Home Equity Line of Credit,...,2006-10-12 00:00:00,None,2006-10-12 00:00:00,11,$206.42,"$-1,898.32",$0.00,$0.00,$0.00,$0.00
342,523319003,Account Mapped Trxs 10 (90.91%),NaN,NaN,NaN,NaN,NaN,NaN,HSUFXNKONE6HHUIFNTLO-0001-Mortgage Loans,Home Equity Line of Credit,...,2006-10-12 00:00:00,None,2006-10-12 00:00:00,11,$206.42,"$-1,898.32",$0.00,$0.00,$0.00,$0.00
